## Problem 8
#### WWSD

In [ ]:
>>> from scheme import *
>>> env = create_global_frame()
>>> eval_all(Pair(2, nil), env)
Choose the number of the correct choice:
0) SchemeError
1) 2

Ans: 1

In [ ]:
>>> eval_all(Pair(4, Pair(5, nil)), env)
Choose the number of the correct choice:
0) (4 5)
1) SchemeError
2) 4
3) 5

Ans: 3

In [ ]:
>>> from scheme import *
>>> env = create_global_frame()
>>> lst = Pair(1, Pair(2, Pair(3, nil)))
>>> eval_all(lst, env)
3
>>> lst     # The list should not be mutated!
Pair(1, Pair(2, Pair(3, nil)))

In [ ]:
scm> (begin (+ 2 3) (+ 5 6))
11
scm> (begin (define x 3) x)
3

In [ ]:
scm> (begin 30 '(+ 2 2))
Choose the number of the correct choice:
0) '(+ 2 2)
1) 30
2) (+ 2 2)
3) 4

Ans: 2

In [ ]:
scm> (define x 0)
x
scm> (begin (define x (+ x 1)) 42 (define y (+ x 1)))
y
scm> x
1
scm> y
2

#### Strategy

Through trial and error of the ok test, we know the following information,

If we leave the code as it was:

In [ ]:
def eval_all(expressions, env):
    return scheme_eval(expressions.first, env)

>>> from scheme import *
>>> env = create_global_frame()
>>> eval_all(Pair(2, nil), env)
2
>>> eval_all(Pair(4, Pair(5, nil)), env)
4

# Error: expected
#     5
# but got
#     4

The code passes the first test, but not the second test. On the second test, we need to return the last expression, which is `5`.

We can access the expression after `expressions.first` using `expressions.second`.

In [ ]:
def eval_all(expressions, env):
    return expressions.second

>>> from scheme import *
>>> env = create_global_frame()
>>> eval_all(Pair(2, nil), env)
nil

# Error: expected
#     2
# but got
#     nil

As can be seen above, with `expressions.second`, we can access the `nil`!

The tests above tell us that we might need to use an `if` statement:

1. If the `expressions.second` is a `nil` object, then we return the result of evaluating the `.first` of the expression we are currently selecting
2. Otherwise, we evaluate the `.first` expression, then move on to recursive call `eval_all` on the `expression.second`.

In [ ]:
if expressions.second is nil:
    return scheme_eval(expressions.first, env)
else:
    scheme_eval(expressions.first, env)
    return eval_all(expressions.second, env)


>>> from scheme import *
>>> env = create_global_frame()
>>> eval_all(Pair(2, nil), env)
2
>>> eval_all(Pair(4, Pair(5, nil)), env)
5
>>> eval_all(nil, env) # return None (meaning undefined)
Traceback (most recent call last):
  File "/home/ronald/Documents/cs61a/Projects/Project 4 - Scheme/scheme.py", line 62, in eval_all
    if expressions.second is nil:
AttributeError: 'nil' object has no attribute 'second'

We are close! Turns out we need a base case in case the expression is initially a `nil` by itself! If the `expressions` is `nil`, then we just return `None`.

The final implementation looks like the following,

In [ ]:
def eval_all(expressions, env):
    if expressions is nil:
        return None
    elif expressions.second is nil:
        return scheme_eval(expressions.first, env)
    else:
        scheme_eval(expressions.first, env)
        return eval_all(expressions.second, env)

## Problem 9
#### WWSD

In [ ]:
scm> (lambda (x y) (+ x y))
(lambda (x y) (+ x y))

#### Strategy

If we return `expressions.first`, or `formal`, we'll obtain the following,

In [ ]:
(x y)

And if we return `expressions.second`, we'll obtain,

In [ ]:
(+ x y)

The problem description describes that the `LambdaProcedure` takes:

1. A list of `formals` (parameter names)
2. a `body` of expressions to evaluate.

Now if we look at the `LambdaProcedure` class:

In [ ]:
class LambdaProcedure(Procedure):
    
    def__init__(self, formals, body, env):
        self.formals = formals
        self.body = body
        self.env = env

We can see that the code perfectly fits! `.formals` is the `expressions.first`, while `.body` is `expressions.second`.

In [ ]:
def do_lambda_form(expressions, env):
    check_form(expressions, 2)
    formals = expressions.first
    check_formals(formals)
    return LambdaProcedure(formals, expressions.second, env)

## Problem 10
#### WWSD

In [ ]:
scm> (define (f x y) (+ x y))
f
scm> f
Choose the number of the correct choice:
0) (lambda (f x y) (+ x y))
1) (define f (lambda (x y) (+ x y)))
2) (f (x y) (+ x y))
3) (lambda (x y) (+ x y))

Ans: 3

#### Strategy

Based on the `ok` test and the `WWSD` above, we want to bind `f` to a `LambdaProcedure`,

In [ ]:
(lambda (x y) (+ x y))

We can obtain the symbol `f` by returning the `target.first`.

In [ ]:
def do_define_form(expressions, env):
    ...
    ...
    elif isinstance(target, Pair) and scheme_symbolp(target.first):
        return target.first
    
scm> (define (f x y) (+ x y))
f
scm> f
# Error: unknown identifier: f

As we can see above, we pass the first test that defining a user-defined procedure should return the name that is bound to the procedure.

Recall that to bind a `name` to a value or expression, we can use the `define` method of a `Frame` class. 

In [ ]:
env.define(target.first, ..., env)

Where the first argument is the name that we want to bind the expression to. However, if we look at the expression,

In [ ]:
elif isinstance(target, Pair) and scheme_symbolp(target.first):
    return expressions

scm> (define (f x y) (+ x y))
((f x y) (+ x y))

The `expressions` consists of the following,

In [ ]:
((f x y) (+ x y))

Since `f` is within the `expressions`, we can't assign `f` to a `do_lambda_form`. `do_lambda_form` can be used if the `expressions` is,

In [ ]:
((x y) (+ x y))

Instead, we use a `LambdaProcedure`, where the first argument is the parameters `(x y)`, the second argument is the body of the expression to evaluate `(+ x y)`, and the last argument is the parent frame `env`. The parameters `(x y)` can be obtained by `target.second`, while the body `((+ x y))` can be obtained by `expressions.second`.

In [ ]:
elif isinstance(target, Pair) and scheme_symbolp(target.first):
    print(target.second)
    print(expressions.second)
    
scm> (define (f x y) (+ x y))
(x y)
((+ x y))

# Error: expected
#     f
# but got
#     (x y)
#     ((+ x y))

Thus, the implementation would be as the following,

In [ ]:
elif isinstance(target, Pair) and scheme_symbolp(target.first):
    env.define(target.first, LambdaProcedure(target.second, expressions.second, env))
    return target.first

## Problem 11

#### WWSD

In [ ]:
>>> from scheme import *
>>> global_frame = create_global_frame()
>>> formals = Pair('a', Pair('b', Pair('c', nil)))
>>> vals = Pair(1, Pair(2, Pair(3, nil)))
>>> frame = global_frame.make_child_frame(formals, vals)
>>> global_frame.lookup('a') # Type SchemeError if you think this errors
SchemeError
>>> frame.lookup('a')        # Type SchemeError if you think this errors
1
>>> frame.lookup('b')        # Type SchemeError if you think this errors
2
>>> frame.lookup('c')        # Type SchemeError if you think this errors
3

In [ ]:
>>> from scheme import *
>>> global_frame = create_global_frame()
>>> frame = global_frame.make_child_frame(nil, nil)
>>> frame.parent is global_frame
True

#### Strategy

The first step is to create a new `Frame` object with `self` as the `parent`. This can be easily done by the following,

In [ ]:
new = Frame(self)

The next step is to bind each `formals` parameter to its corresponding argument `value` in the newly created frame. However, if the number of `formals` and `values` do not match, we raise a `SchemeError`.

From the `ok` test, if we try to see what `formals` and `vals` look like, 

In [ ]:
def make_child_frame(self, formals, vals):
    print(formals)
    print(vals)
    
>>> from scheme import *
>>> global_frame = create_global_frame()
>>> formals = Pair('a', Pair('b', Pair('c', nil)))
>>> vals = Pair(1, Pair(2, Pair(3, nil)))
>>> frame = global_frame.make_child_frame(formals, vals)
(a b c)
(1 2 3)

# Error: expected

# but got
#     (a b c)
#     (1 2 3)

As we can see, `formals` and `vals` are constructed through the `Pair` class, but represented as a Scheme list. If we look at the definition of `Pair` class in `scheme_reader.py` file, 

In [ ]:
class Pair(object):
    ...
    ...
    def __len__(self):
    n, second = 1, self.second
    while isinstance(second, Pair):
        n += 1
        second = second.second
    if second is not nil:
        raise TypeError('length attempted on improper list')
    return n

The `Pair` class has a `__len__` method, which computes the length of a `Pair` class! We can use this method as the following,

In [ ]:
def make_child_frame(self, formals, vals):
        print(formals.__len__())
        print(vals.__len__())

>>> from scheme import *
>>> global_frame = create_global_frame()
>>> formals = Pair('a', Pair('b', Pair('c', nil)))
>>> vals = Pair(1, Pair(2, Pair(3, nil)))
>>> frame = global_frame.make_child_frame(formals, vals)
3
3

# Error: expected

# but got
#     3
#     3

Now that we know how to obtain the length of `formals` and `vals`, we set a condition that if the length of `formals` and `vals` are not the same, we raise a `SchemeError`.

In [ ]:
if formals.__len__() != vals.__len__():
    raise SchemeError

Otherwise, we use the `define` method on the newly created `new` frame by assigning the parameter `formals.first` to the value `vals.first`, then we shift both `formals` and `vals` until we reach `nil`.

In [ ]:
while formals is not nil:
    new.define(formals.first, vals.first)
    formals, vals = formals.second, vals.second

Then we return that new frame. The implementation looks like the following,

In [ ]:
def make_child_frame(self, formals, vals):
    new = Frame(self)
    if formals.__len__() != vals.__len__():
        raise SchemeError
    while formals is not nil:
        new.define(formals.first, vals.first)
        formals, vals = formals.second, vals.second
    return new

## Problem 12
#### WWSD

In [ ]:
scm> (define (outer x y)
....   (define (inner z x)
....     (+ x (* y 2) (* z 3)))
....   (inner x 10))
outer
scm> (outer 1 2)
17

Above is similar to the following,

In [ ]:
(define (outer x y)
 (define (inner a b)
  (+ b (* y 2) (* a 3))
 (inner x 10)

As we can see, we set `10` to be the value for `b`, while we set `a` to be the argument `x`. This is how it works with the previous cell, where we have `x` in inner scope and `x` in outer scope. 

In [ ]:
scm> (define (outer-func x y)
....   (define (inner z x)
....     (+ x (* y 2) (* z 3)))
....   inner)
outer-func
scm> ((outer-func 1 2) 1 10)
17

#### Strategy

Following the description given by the problem statement, BEWARE that the given `env` is not the frame in which the procedure is **defined**, but instead the frame in which the procedure is **called**.

A common mistake is that we implement the `make_call_frame` as the following,

In [ ]:
def make_call_frame(self, args, env):
    return env.make_child_frame(self.formals, args)

In the implementation above, we are calling the `make_child_frame` method on the environment in which the procedure is called! Instead, we want to call the method on the environment in which the procedure is defined, which is `self.env`.

In [ ]:
def make_call_frame(self, args, env):
    return self.env.make_child_frame(self.formals, args)